In [1]:
!pip install langchain
!pip install torch
!pip install accelerate
!pip install sentence-transformers
!pip install streamlit
!pip install streamlit-chat
!pip install faiss-cpu
!pip install tiktoken
!pip install huggingface-hub
!pip install pypdf
!pip install llama-cpp-python
!pip install -U langchain-community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 975.5/975.5 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.4/337.4 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.5/127.5 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 10.4 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux

In [2]:
from langchain.chains import ConversationalRetrievalChain
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import LlamaCpp
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.memory import ConversationBufferMemory
from langchain.document_loaders import PyPDFLoader
import os
import tempfile
import openpyxl
from openpyxl import Workbook,load_workbook
from langchain.llms import DeepInfra
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer, util
import spacy

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [25]:
import re

In [3]:
def initialize_session_state():
    session_state = {
        "history": [],
        "generated": ["Hello! Ask me anything about 🤖"],
        "past": ["Hey! 👋"]
    }
    return session_state

In [4]:
def conversation_chat(query, chain, session_state):
    #session_state["history"] = []
    session_state = initialize_session_state()
    result = chain({"question": query, "chat_history": session_state["history"]})
    session_state["history"].append((query, result["answer"]))
    return result["answer"]

In [5]:
def display_chat_history(chain, session_state, questions):
    workbook = Workbook()
    sheet = workbook.active
    sheet.append(["Question", "Answer", "Relevance Score", "Status"])

    for question, repeats in questions:
        for _ in range(repeats):
            output = conversation_chat(question, chain, session_state)
            score, status = check_relevance(question, output)
            sheet.append([question, output, score, status])
            print(f"User: {question}\nBot: {output}\nScore: {score}\nStatus: {status}")

    workbook.save("ChatHistory.xlsx")
    print("Chat history saved to ChatHistory.xlsx")



In [6]:
os.environ["DEEPINFRA_API_TOKEN"] = "wUp54XhDTB1htN0ios9N8JMSg9rKpedN"

In [7]:

def create_conversational_chain(vector_store):
    local_llm = DeepInfra(model_id="mistralai/Mistral-7B-Instruct-v0.1")

    local_llm.model_kwargs = {
        "streaming": True,
        "temperature": 0.5,
        "top_p": 0.7,
        "verbose": True,
        "n_ctx": 4096,
    }

    memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
    chain = ConversationalRetrievalChain.from_llm(
        llm=local_llm,
        chain_type="stuff",
        retriever=vector_store.as_retriever(search_kwargs={"k": 2}),
        memory=memory
    )
    return chain



In [28]:
# Load spaCy model for named entity recognition
nlp = spacy.load("en_core_web_sm")

# Define functions for computing similarities
def compute_tfidf_similarity(text1, text2):
    vectorizer = TfidfVectorizer().fit_transform([text1, text2])
    vectors = vectorizer.toarray()
    return cosine_similarity([vectors[0]], [vectors[1]])[0][0]

def compute_bert_similarity(text1, text2):
    model = SentenceTransformer('all-MiniLM-L6-v2')
    embeddings1 = model.encode(text1, convert_to_tensor=True)
    embeddings2 = model.encode(text2, convert_to_tensor=True)
    return util.pytorch_cos_sim(embeddings1, embeddings2).item()

def compute_combined_similarity(text1, text2, weight_tfidf=0.3, weight_bert=0.7):
    tfidf_similarity = compute_tfidf_similarity(text1, text2)
    bert_similarity = compute_bert_similarity(text1, text2)
    combined_similarity = weight_tfidf * tfidf_similarity + weight_bert * bert_similarity
    return combined_similarity

def contains_named_entity(text):
    doc = nlp(text)
    return any(ent.label_ in ["PERSON", "ORG", "GPE"] for ent in doc.ents)

def check_relevance(question, generated_answer):
    # Calculate similarity score and relevance
    similarity_score = compute_combined_similarity(question, generated_answer)

    # Additional check for named entities
    if contains_named_entity(question) and contains_named_entity(generated_answer):
        relevance = "Relevant"
    else:
        relevance = "Relevant" if similarity_score >= 0.4 else "Not Relevant"

    return similarity_score, relevance



In [33]:

#many questions added manually
def main():
    session_state = initialize_session_state()
    print("ChatBot using Mistral-7B-Instruct LLM :books:")

    uploaded_files = ["/content/2023.pdf"]

    if uploaded_files:
        text = []
        for file_path in uploaded_files:
            with open(file_path, "rb") as f:
                file_contents = f.read()
            file_extension = os.path.splitext(file_path)[1]
            with tempfile.NamedTemporaryFile(delete=False) as temp_file:
                temp_file.write(file_contents)
                temp_file_path = temp_file.name
                print("loading: ", file_path)
            loader = None
            if file_extension == ".pdf":
                loader = PyPDFLoader(temp_file_path)

            if loader:
                text.extend(loader.load())
                os.remove(temp_file_path)

        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=10000, chunk_overlap=20
        )

        text_chunks = text_splitter.split_documents(text)

        embeddings = HuggingFaceEmbeddings(
            model_name="sentence-transformers/all-MiniLM-L6-v2",
            model_kwargs={"device": "cpu"},
        )

        vector_store = FAISS.from_documents(text_chunks, embedding=embeddings)

        chain = create_conversational_chain(vector_store)

        questions = []
        while True:
            user_input = input("Enter your question (or type 'done' to finish): ")
            if user_input.lower() == 'done':
                break
            repeats = int(input(f"How many times to repeat '{user_input}': "))
            questions.append((user_input, repeats))

        display_chat_history(chain, session_state, questions)

if __name__ == "__main__":
    main()


ChatBot using Mistral-7B-Instruct LLM :books:
loading:  /content/2023.pdf
Enter your question (or type 'done' to finish): What do you know about ave maria convent?
How many times to repeat 'What do you know about ave maria convent?': 10
Enter your question (or type 'done' to finish): done


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


User: What do you know about ave maria convent?
Bot:  I don't know about Ave Maria convent.
Score: 0.8063467191679605
Status: Relevant


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


User: What do you know about ave maria convent?
Bot:   Ave Maria convent is not mentioned in the given context.

51  
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 

Score: 0.5790043612226597
Status: Relevant


...

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


User: What do you know about ave maria convent?
Bot:   Ave Maria convent is not mentioned in the given context. 

51  
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
Score: 0.5790043612226597
Status: Relevant


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


User: What do you know about ave maria convent?
Bot:   Ave Maria convent is not mentioned in the given context.

51  
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 

Score: 0.5790043612226597
Status: Relevant


KeyboardInterrupt: 

In [10]:
# Load spaCy model for named entity recognition
nlp = spacy.load("en_core_web_sm")

# Define functions for computing similarities
def compute_tfidf_similarity(text1, text2):
    vectorizer = TfidfVectorizer().fit_transform([text1, text2])
    vectors = vectorizer.toarray()
    return cosine_similarity([vectors[0]], [vectors[1]])[0][0]

def compute_bert_similarity(text1, text2):
    model = SentenceTransformer('all-MiniLM-L6-v2')
    embeddings1 = model.encode(text1, convert_to_tensor=True)
    embeddings2 = model.encode(text2, convert_to_tensor=True)
    return util.pytorch_cos_sim(embeddings1, embeddings2).item()

def compute_combined_similarity(text1, text2, weight_tfidf=0.3, weight_bert=0.7):
    tfidf_similarity = compute_tfidf_similarity(text1, text2)
    bert_similarity = compute_bert_similarity(text1, text2)
    combined_similarity = weight_tfidf * tfidf_similarity + weight_bert * bert_similarity
    return combined_similarity

def contains_named_entity(text):
    doc = nlp(text)
    return any(ent.label_ in ["PERSON", "ORG", "GPE"] for ent in doc.ents)

# Load questions and answers from Excel file
input_file = '/content/Book2.xlsx'
output_file = 'similarity_results.xlsx'

df = pd.read_excel(input_file)

# Ensure the input file has columns 'Question' and 'Answer'
if 'Question' not in df.columns or 'Answer' not in df.columns:
    raise ValueError("Input Excel file must contain 'Question' and 'Answer' columns.")

results = []
threshold = 0.4  # Adjust as needed

# Iterate through the questions and answers
for index, row in df.iterrows():
    question = row['Question']
    generated_answer = row['Answer']

    similarity_score = compute_combined_similarity(question, generated_answer)

    # Additional check for named entities
    if contains_named_entity(question) and contains_named_entity(generated_answer):
        relevance = "Relevant"
    else:
        relevance = "Relevant" if similarity_score >= threshold else "Not Relevant"

    results.append({
        'Question': question,
        'Generated Answer': generated_answer,
        'Similarity Score': similarity_score,
        'Relevance': relevance
    })

# Create a DataFrame for the results
results_df = pd.DataFrame(results)

# Write the results to an Excel file
results_df.to_excel(output_file, index=False)

print(f"Results have been written to {output_file}")


FileNotFoundError: [Errno 2] No such file or directory: '/content/Book2.xlsx'